In [ ]:
!pip install subword-nmt evaluate
!! pip install open-metric-learning
!! pip install pillow==9.0.0
!pip install natasha
pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_hard_tri import HardTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset

from IPython.display import clear_output
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
from tqdm import tqdm

from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from vocab import Vocab
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ExponentialLR
from transformers import (DataCollatorForSeq2Seq, AutoTokenizer,
                          AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)

from natasha import (
    Segmenter,
    MorphVocab,
    LOC,
    AddrExtractor    
)

import datasets
import evaluate

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [38]:
df1 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_1.csv').drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_2.csv').drop('Unnamed: 0', axis=1)
df3 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_3.csv').drop('Unnamed: 0', axis=1)
df4 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_4.csv').drop('Unnamed: 0', axis=1)
# df5 = pd.read_csv('./datasets/dataset_5.csv').drop('Unnamed: 0', axis=1)

df = pd.concat((df1, df2, df3, df4)).dropna()

In [39]:
building = pd.read_csv('/kaggle/input/cp-dataset/additional_data/building_20230808.csv')
area = pd.read_csv('/kaggle/input/cp-dataset/additional_data/area_20230808.csv')
areatype = pd.read_csv('/kaggle/input/cp-dataset/additional_data/areatype_20230808.csv')
district = pd.read_csv('/kaggle/input/cp-dataset/additional_data/district_20230808.csv')
geonim = pd.read_csv('/kaggle/input/cp-dataset/additional_data/geonim_20230808.csv')
geonimtype = pd.read_csv('/kaggle/input/cp-dataset/additional_data/geonimtype_20230808.csv')
prefix = pd.read_csv('/kaggle/input/cp-dataset/additional_data/prefix_20230808.csv')
subrf = pd.read_csv('/kaggle/input/cp-dataset/additional_data/subrf_20230808.csv')
town = pd.read_csv('/kaggle/input/cp-dataset/additional_data/town_20230808.csv')

/tmp/ipykernel_198/3938848616.py:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv('/kaggle/input/cp-dataset/additional_data/building_20230808.csv')


In [40]:
df = pd.concat((building[['id', 'full_address']].rename(columns={'full_address': 'address', 'id': 'target_building_id'}),
                building[['id', 'short_address']].rename(columns={'short_address': 'address', 'id': 'target_building_id'}),
                df[['address', 'target_building_id']],
                df[['target_address', 'target_building_id']].rename(columns={'target_address': 'address'})
               ))

In [4]:
train = pd.read_csv('train.csv').drop('Unnamed: 0', axis=1)
train = pd.concat((train, train))
df = train.copy()

In [6]:
tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

with open('data.txt', 'w') as f:
    for line in tqdm(df['address']):
        f.write(tokenize(line) + '\n')

100%|██████████| 1137290/1137290 [00:13<00:00, 82755.00it/s]


In [7]:
# строим и применяем bpe кодирование
learn_bpe(open('data.txt', encoding='utf-8'), open('bpe_rules', 'w', encoding='utf-8'), num_symbols=8000)
bpe = BPE(open('./bpe_rules', encoding='utf-8'))

with open('train.bpe', 'w', encoding='utf-8') as f_out:
    for line in tqdm(open('data.txt', encoding='utf-8')):
        f_out.write(bpe.process_line(line.strip()) + '\n')

100%|██████████| 8000/8000 [00:05<00:00, 1597.16it/s]
1137290it [00:44, 25497.36it/s]


In [8]:
data = np.array(open('./train.bpe', encoding='utf-8').read().split('\n'))[:-1]

In [9]:
data = data[:, np.newaxis]
target = df['target_building_id'].to_numpy()[:, np.newaxis].astype(int)

In [10]:
data = np.concatenate((data, target), axis=1)

In [11]:
train_inp, dev_inp, train_target, dev_target = train_test_split(data[:, 0], data[:, 1].astype(int),
                                                                test_size=1000,
                                                                random_state=42)

In [12]:
inp_voc = Vocab.from_lines(train_inp)

In [57]:
class CPDataset(Dataset): 
    def __init__(self, inp, target):
        self.inp = inp
        self.target = target
        
        self.inp_voc = Vocab.from_lines(self.inp)
        
    def __len__(self):
        return len(self.inp)
        
    def __getitem__(self, idx):
        return {'address': self.inp[idx],
                'id': int(self.target[idx])}

def batch_collate_fn(batch):
    inps = [item['address'] for item in batch]
    labels = np.array([item['id'] for item in batch])
    return {'address': train_dataset.inp_voc.to_matrix(inps),
                'id': labels}

In [159]:
class BaseModel(nn.Module):
    def __init__(self, inp_voc, emb_size=64, hid_size=128):
        """
        Базовая модель encoder-decoder архитектуры
        """
        super().__init__() 

        self.inp_voc = inp_voc
        self.hid_size = hid_size
        
        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)
        
        self.act = nn.ReLU()
        self.fc = nn.Linear(hid_size, hid_size)
        
    def forward(self, inp):
        """ Сначала примените  encode а затем decode"""
        ans = self.encode(inp)
        return ans


    def encode(self, inp, **flags):
        """
        Считаем скрытое состояние, которое будет начальным для decode
        :param inp: матрица входных токенов
        :returns: скрытое представление с которого будет начинаться decode
        """
        inp_emb = self.emb_inp(inp)
        batch_size = inp.shape[0]
        
        enc_seq, [last_state_but_not_really] = self.enc0(inp_emb)
        # enc_seq: [batch, time, hid_size], last_state: [batch, hid_size]
        
        # последний токен, не последние на самом деле, так как мы делали pading, чтобы тексты были
        # одинакового размер, поэтому подсчитать длину исходного предложения не так уж тривиально
        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        last_state = enc_seq[torch.arange(len(enc_seq)), lengths]
        # ^-- shape: [batch_size, hid_size]
        
        out = self.fc(last_state)
        return last_state

In [160]:
class Fitter(object):
    def __init__(
        self,
        model,
        opt,
        loss,
        miner,
        train_dataloader,
        scheduler,
#         dev_dataloader,
        batch_size = 32,
        sheduler_steps = 10,
        n_epochs = 10
    ):

        self.model = model
        self.n_epochs = n_epochs
        self.sheduler_steps = sheduler_steps

        self.opt = opt
        self.loss = loss
        self.scheduler = scheduler
        self.train_dataloader = train_dataloader
        self.miner = miner
#         self.dev_dataloader = dev_dataloader
        
        self.model.to(device)
        self.metrics = {'train_loss': [], 'accuracy': [], 'train_batch_loss': []}

    def fit(self):
        # Turn on training
        self.model.train(True)
        
        # Fit
        for epoch in range(self.n_epochs):
            train_loss = 0
            self.metrics['train_batch_loss'] = []
            for i, batch in enumerate(tqdm(
                self.train_dataloader, desc='Работаем Братья')):
                
                
#                 print(batch['address'])
                targets = torch.tensor(batch['id'], device=device)
                self.opt.zero_grad()
                inp = torch.tensor(batch['address'], device=device)
                embeddings = self.model(inp)
                hard_pairs = self.miner(embeddings, targets)
                loss = self.loss(embeddings, targets, hard_pairs)
                loss.backward()
                self.opt.step()
                train_loss += loss.item()
#                 if (i + 1) % 100 == 0:
#                     self.metrics['train_batch_loss'] += [train_loss / (i + 1)]
#                     clear_output(wait=True)
#                     plt.figure(figsize=(10, 5))
            
#                     plt.subplot(1, 1, 1)
#                     plt.xlabel("Iteration")
#                     plt.ylabel("Train Batch Loss")
#                     plt.grid()
#                     plt.plot(self.metrics['train_batch_loss'])

#                     plt.show()
            
            if epoch % self.sheduler_steps == 1:
                self.scheduler.step()
            
            clear_output(wait=True)  
            self.model.eval()
#             with torch.no_grad():
#                 for i, batch in tqdm(enumerate(
#                 self.dev_dataloader, desc='Работаем Братья', total=len(self.train_dataloader) // self.batch_size)):
                
            self.metrics['train_loss'] += [train_loss / (i+1)]
#                     preds = F.softmax(self.model(self.model.inp_voc.to_matrix(self.dev[0]).to(device)), dim=1).detach().cpu().numpy()
#                     self.metrics['accuracy'] += [roc_auc_score(self.dev[1],
#                                                                   preds,
#                                                                   multi_class='ovr')]
                
            if len(self.metrics['accuracy']) > 2 and self.metrics['accuracy'][-1] > self.metrics['accuracy'][-2]:
                torch.save(self.model, 'rnn_segm.pt')
            
            plt.figure(figsize=(10, 5))
            
            plt.subplot(1, 1, 1)
            plt.xlabel("Iteration")
            plt.ylabel("Train Loss")
            plt.grid()
            plt.plot(self.metrics['train_loss'])

#             plt.subplot(1, 2, 2)
#             plt.xlabel("Iteration")
#             plt.ylabel("ROC-AUC")
#             plt.plot(self.metrics['dev_ROC-AUC'], color="orange")
#             plt.grid()
            
            plt.show()
            
            self.model.train(True)
            
        # Turn off training
        self.model.train(False)

In [161]:
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.regularizers import LpRegularizer
from pytorch_metric_learning import miners, losses
from pytorch_metric_learning import samplers

In [190]:
model = BaseModel(inp_voc).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-6)
scheduler = ExponentialLR(opt, gamma=0.9)
train_dataset = CPDataset(data[:, 0], data[:, 1].astype(int))
# dev_dataset = CPDataset(train_inp, train_target)

# criterion = TripletLossWithMiner(margin=0.1, miner=HardTripletsMiner(), need_logs=True)
criterion = losses.TripletMarginLoss(distance = CosineSimilarity(), 
                                    reducer = ThresholdReducer(high=0.3), 
                                    embedding_regularizer = LpRegularizer())

miner = miners.MultiSimilarityMiner()
# вот эта сука не заводится, скорее всего из-за версии торча, но сверху я поставил чо у них в репе стояло, вроде.... не знаюююю.... не проверял....
train_sampler = samplers.MPerClassSampler(data[:, 1].astype(int),
                                          m=16, batch_size=64,
                                          length_before_new_iter=100000)


# train_sampler = samplers.FixedSetOfTriplets(data[:, 1].astype(int), num_triplets=252)


train_loader = DataLoader(train_dataset,
                          collate_fn=batch_collate_fn,
#                           batch_size=64,
                          batch_sampler=train_sampler
                         )

# dev_sampler = BalanceSampler(dev_target, n_labels=10, n_instances=5)
# dev_loader = DataLoader(dev_dataset, batch_sampler=sampler)

In [192]:
for batch in train_loader:
    print(batch)
    break

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 for batch in train_loader:                                                                   │
│   2 │   print(batch)                                                                             │
│   3 │   break                                                                                    │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:628 in __next__           │
│                                                                                                  │
│    625 │   │   │   if self._sampler_iter is None:                                                │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    627 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  628 │   │   │   data = self._next_data()                                                      │
│    629 │   │   │   self._num_yielded += 1                                                        │
│    630 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    631 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:671 in _next_data         │
│                                                                                                  │
│    668 │                                                                                         │
│    669 │   def _next_data(self):                                                                 │
│    670 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  671 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    672 │   │   if self._pin_memory:                                                              │
│    673 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    674 │   │   return data                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py:58 in fetch             │
│                                                            

In [149]:
sheduler_steps = 1

epochs = 15

fitter =  Fitter(
                model,
                opt,
                criterion,
                miner,
                train_loader,
                scheduler,
#                 dev_loader,
                sheduler_steps = sheduler_steps,
                n_epochs = epochs,
    
)

In [150]:
fitter.fit()

Работаем Братья:   0%|          | 0/99968 [00:16<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 fitter.fit()                                                                                 │
│   2                                                                                              │
│                                                                                                  │
│ in fit:38                                                                                        │
│                                                                                                  │
│    35 │   │   for epoch in range(self.n_epochs):                                                 │
│    36 │   │   │   train_loss = 0                                                                 │
│    37 │   │   │   self.metrics['train_batch_loss'] = []                                          │
│ ❱  38 │   │   │   for i, batch in enumerate(tqdm(                                                │
│    39 │   │   │   │   self.train_dataloader, desc='Работаем Братья')):                           │
│    40                                                                                            │
│    41                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/tqdm/std.py:1178 in __iter__                             │
│                                                                                                  │
│   1175 │   │   time = self._time                                                                 │
│   1176 │   │                                                                                     │
│   1177 │   │   try:                                                                              │
│ ❱ 1178 │   │   │   for obj in iterable:                                                          │
│   1179 │   │   │   │   yield obj                                                                 │
│   1180 │   │   │   │   # Update and possibly print the progressbar.                              │
│   1181 │   │   │   │   # Note: does not call self.update(1) for speed optimisation.              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:628 in __next__           │
│                                                                                                  │
│    625 │   │   │   if self._sampler_iter is None:                                                │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/py

In [107]:
df1 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_1.csv').drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_2.csv').drop('Unnamed: 0', axis=1)
df3 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_3.csv').drop('Unnamed: 0', axis=1)
df4 = pd.read_csv('/kaggle/input/cp-dataset/datasets/dataset_4.csv').drop('Unnamed: 0', axis=1)
# df5 = pd.read_csv('./datasets/dataset_5.csv').drop('Unnamed: 0', axis=1)

df = pd.concat((df1, df2, df3, df4)).dropna()
df = df.reset_index(drop=True)

In [108]:
from sklearn.metrics.pairwise import cosine_similarity

In [138]:
train_dataset.inp_voc.to_matrix([df.loc[0, 'address']])

tensor([[0, 2, 2, 2, 2, 2, 2, 1]])

In [141]:
model.eval()
inp1 = train_dataset.inp_voc.to_matrix([df.loc[0, 'address']]).to(device)
sims = dict()
with torch.no_grad():
    emb1 = model(inp1).clone().detach().cpu()
    for i in tqdm(range(len(df))):
        inp2 = train_dataset.inp_voc.to_matrix([df.loc[i, 'target_address']]).to(device)
        emb2 = model(inp2).cpu()
        sims[df.loc[i, 'target_address']] = cosine_similarity(emb1, emb2)

100%|██████████| 36002/36002 [00:39<00:00, 910.04it/s]


In [145]:
df.loc[0, 'address']

'196641, Санкт-Петербург г, Школьная ул, д.5'

In [144]:
sorted(sims.items(), key=lambda x: -x[1])

[('192007, г.Санкт-Петербург, Тамбовская улица, дом 7, корпус 2, строение1',
  array([[0.9917274]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СНТ п.Ольгино Нижний сад, Берёзовая линия, дом 27, литера А',
  array([[0.9886947]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СНТ п.Ольгино Нижний сад, Берёзовая линия, дом 24, литера А',
  array([[0.9886947]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СНТ п.Ольгино Нижний сад, Берёзовая линия, дом 23, литера А',
  array([[0.9886947]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СНТ п.Ольгино Нижний сад, Берёзовая линия, дом 2, литера А',
  array([[0.9886947]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СНТ п.Ольгино Нижний сад, Берёзовая линия, дом 53, литера А',
  array([[0.9886947]], dtype=float32)),
 ('Санкт-Петербург, муниципальный округ Лахта - Ольгино, тер. СН

In [110]:
emb1

tensor([[-0.1435, -0.0729,  0.0007,  ..., -0.0175, -0.0668, -0.0435],
        [-0.0331, -0.0503, -0.0099,  ..., -0.0437,  0.0363,  0.0015],
        [-0.1678, -0.1076, -0.0314,  ..., -0.0284,  0.0400, -0.0012],
        ...,
        [-0.1149, -0.0541,  0.0146,  ..., -0.0596, -0.0099,  0.0219],
        [-0.1090, -0.0512, -0.0119,  ..., -0.0252, -0.0020, -0.0053],
        [-0.2302, -0.2324, -0.0067,  ..., -0.0306, -0.0065, -0.0003]])

In [113]:
df.loc[0, 'address']

'196641, Санкт-Петербург г, Школьная ул, д.5'

In [114]:
df.loc[0, 'target_address']

'Санкт-Петербург, город Павловск, Пязелево, Школьная улица, дом 5, литера А'

In [115]:
train_dataset.inp_voc.to_matrix([df.loc[0, 'address']])

tensor([[0, 2, 2, 2, 2, 2, 2, 1]])